# Setup

In [1]:
import sys
sys.path.append("../../.")

NThreads = 16

In [2]:
from Construction.Components import Component, get_XORn, get_COPYn
from Construction.CompoundFunction import CompoundFunction, INPUT_ID, OUTPUT_ID
from Construction.IteratedCipher import construct_iterated_cipher

In [3]:
class lshiftAND(Component):

    def __init__(self, n, l):
        super().__init__(n, n)
        self.n = n
        self.l = l


    @staticmethod
    def lshift(x, s, n):
        return ((x & (2**(n-s)-1)) << s) | (x >> (n-s))

    def __call__(self, v):
        r = 2**self.n - 1
        for s in self.l:
            r &= lshiftAND.lshift(v, s, self.n)
        return r
    
    def compute_parity_propagation_model(self, input_key_mask, output_key_mask): #ignoring input and output key mask
        clauses = []
        output_vars = list(range(self.n+1, 2*self.n+1))
        for i in range(self.n):
            clauses.append((-(i+1), ) + tuple(output_vars[(i+s) % self.n] for s in self.l))
            for s in self.l:
                clauses.append((i+1, -output_vars[(i+s) % self.n]))
        return tuple(clauses)

    def compute_UT_propagation_model(self):
        return self.compute_parity_propagation_model(0, 0), 0

In [4]:
# builds the simon round function, also allows for variations on the rotations, such as simeck
def generate_SIMON_round_function(n, var=(8,1,2)):
    f = CompoundFunction(3*n, 2*n) # last 16 bits of input are key material
    cids = [f.add_component(get_COPYn(3)) for _ in range(n)]
    fid = f.add_component(lshiftAND(n, var[:2]))
    xids = [f.add_component(get_XORn(4)) for _ in range(n)]
    for i in range(n):
        f.connect_components(INPUT_ID, i, xids[i], 0)
        f.connect_components(INPUT_ID, n+i, cids[i], 0)
        f.connect_components(INPUT_ID, 2*n+i, xids[i], 1)
        f.connect_components(cids[i], 0, OUTPUT_ID, i)
        f.connect_components(cids[i], 1, fid, i)
        f.connect_components(cids[(i-var[2]) % n], 2, xids[i], 2)
        f.connect_components(fid, i, xids[i], 3)
        f.connect_components(xids[i], 0, OUTPUT_ID, i+n)
    return f

In [5]:
def build_SIMON(n, r, var=(8,1,2)):
    rf = generate_SIMON_round_function(n, var)
    f = CompoundFunction(2*n, 2*n)
    state = [(INPUT_ID, i) for i in range(2*n)]
    for _ in range(r):
        cid = f.add_component(rf)
        for i in range(n):
            f.connect_components(*state[i], cid, i)
            f.connect_components(*state[i+n], cid, i+n)
            f.connect_to_key(cid, i+2*n)
            state[i] = (cid, i)
            state[i+n] = (cid, i+n)
    for i in range(2*n):
        f.connect_components(*state[i], OUTPUT_ID, i)
    return f

# build simon without the first round, but with the first key addition. (This is the version we will analyse)
def build_SIMON_var(n, r, var=(8,1,2)): 
    rf = generate_SIMON_round_function(n, var)
    f = CompoundFunction(2*n, 2*n)
    state = [(INPUT_ID, i) for i in range(2*n)]
    xids = [f.add_component(get_XORn(2)) for _ in range(n)]
    for i in range(n):
        f.connect_components(*state[n+i], xids[i], 0)
        f.connect_to_key(xids[i], 1)
        state[n+i] = (xids[i], 0)
    for _ in range(r):
        cid = f.add_component(rf)
        for i in range(n):
            f.connect_components(*state[i], cid, i)
            f.connect_components(*state[i+n], cid, i+n)
            f.connect_to_key(cid, i+2*n)
            state[i] = (cid, i)
            state[i+n] = (cid, i+n)
    for i in range(2*n):
        f.connect_components(*state[i], OUTPUT_ID, i)
    return f

In [6]:
from Modelling.UT_Trails import get_divisibility_no_trail, get_divisibility_no_key_dependent_trail, get_divisibility_no_key_dependence, get_divisibility_of_key_dependence_no_trail

In [7]:
from multiprocessing import Pool
from functools import partial
from time import time
def test(n, r, u, strong=False): # test the divisibility of the output bits, return the minimum and maximum divisibility for both halves
    f = build_SIMON_var(n, r)
    model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
    with Pool(NThreads) as pool:
        if strong:
            return list(pool.map(partial(get_divisibility_no_key_dependent_trail, model, input_vars, output_vars, key_vars, count_vars, u), [1<<i for i in range(2*n)], 1))
        return list(pool.map(partial(get_divisibility_no_trail, model, input_vars, output_vars, key_vars, count_vars, u), [1<<i for i in range(2*n)], 1))

# Properties

 ## Simon-32

In [8]:
l = [(16, 6, 2**17-1), (16, 7, 2**25-1), (16, 8, 2**29-1), (16, 9, 2**31-1), (16, 10, 2**31-1), (16, 11, 2**31-1), (16, 12, 2**31-1), (16, 13, 2**31-1), (16, 14, 2**31-1)]
for t in l:
    start = time()
    print(2*t[0], t[1]+1, hex(t[2]), test(*t, strong=True), time()-start)

32 7 0x1ffff [7, 6, 7, 6, 6, 6, 7, 7, 7, 7, 6, 6, 6, 7, 7, 7, 3, 3, 2, 2, 3, 3, 3, 3, 3, 2, 3, 3, 2, 2, 3, 3] 7.574222803115845
32 8 0x1ffffff [6, 6, 6, 7, 7, 7, 6, 6, 6, 6, 7, 7, 7, 7, 6, 5, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3] 16.92855215072632
32 9 0x1fffffff [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3] 34.46174430847168
32 10 0x7fffffff [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3] 45.0351505279541
32 11 0x7fffffff [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 14.748093128204346
32 12 0x7fffffff [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 5.279537677764893
32 13 0x7fffffff [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 3.6914780139923096
32 14 0x7fffffff [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0

## Simon-48

In [10]:
l = [(24, 6, 2**17-1), (24, 7, 2**29-1), (24, 8, 2**39-1), (24, 9, 2**44-1), (24, 10, 2**46-1), (24, 11, 2**47-1), (24, 12, 2**47-1), (24, 13, 2**47-1), (24, 14, 2**47-1), (24, 15, 2**47-1), (24, 16, 2**47-1)]
for t in l:
    start = time()
    print(2*t[0], t[1]+1, hex(t[2]), test(*t, strong=False), time()-start)

48 7 0x1ffff [10, 9, 9, 8, 9, 9, 9, 10, 9, 9, 8, 8, 8, 8, 7, 7, 7, 7, 7, 6, 6, 7, 8, 9, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 3] 19.37536334991455
48 8 0x1fffffff [10, 9, 9, 8, 8, 8, 9, 9, 9, 8, 8, 7, 6, 6, 7, 7, 8, 9, 9, 8, 8, 8, 8, 9, 4, 4, 4, 4, 3, 3, 3, 4, 4, 4, 3, 3, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4] 61.78871035575867
48 9 0x7fffffffff [8, 8, 8, 8, 7, 7, 8, 8, 8, 8, 8, 7, 7, 7, 7, 6, 7, 7, 7, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4] 1620.1815016269684
48 10 0xfffffffffff [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4] 7170.868550300598
48 11 0x3fffffffffff [5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3] 26207.42924451828
48 12 0x7fffffffffff [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

## Simon-64

In [11]:
t = (32, 6, 2**17-1)
start = time()
print(2*t[0], t[1]+1, hex(t[2]), test(*t, strong=False), time()-start)

64 7 0x1ffff [13, 13, 12, 12, 13, 13, 13, 13, 13, 12, 12, 12, 11, 10, 9, 9, 8, 9, 8, 7, 6, 7, 8, 9, 10, 9, 10, 10, 11, 12, 13, 13, 10, 9, 10, 9, 9, 9, 9, 10, 9, 10, 9, 9, 9, 9, 8, 7, 6, 5, 5, 5, 4, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 9] 100.57196974754333


In [20]:
t = (32, 7, 2**33-1)
start = time()
print(2*t[0], t[1]+1, hex(t[2]), test(*t, strong=False), time()-start)

64 8 0x1ffffffff [12, 13, 13, 13, 13, 13, 13, 12, 13, 12, 12, 12, 12, 12, 12, 13, 12, 13, 12, 12, 12, 13, 13, 13, 13, 12, 12, 13, 13, 13, 13, 13, 7, 6, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 7, 6, 7, 6, 6, 6, 7, 7, 7, 7, 6, 6, 7, 7, 7, 7] 197.04599738121033


In [13]:
t = (32, 16, 2**63-1)
start = time()
print(2*t[0], t[1]+1, hex(t[2]), test(*t, strong=False), time()-start)

64 17 0x7fffffffffffffff [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 139.23929023742676


In [14]:
t = (32, 17, 2**63-1)
start = time()
print(2*t[0], t[1]+1, hex(t[2]), test(*t, strong=False), time()-start)

64 18 0x7fffffffffffffff [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 44.29720687866211


## Simon-96

In [15]:
t = (48, 6, 2**17-1)
start = time()
print(2*t[0], t[1]+1, hex(t[2]), test(*t, strong=False), time()-start)

96 7 0x1ffff [16, 16, 16, 16, 16, 16, 16, 16, 16, 15, 14, 13, 12, 11, 10, 10, 9, 9, 8, 7, 6, 7, 8, 9, 10, 9, 10, 10, 11, 12, 13, 13, 13, 13, 13, 14, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 15, 14, 15, 15, 15, 15, 15, 15, 14, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 9, 10, 9, 10, 10, 11, 12, 13, 13, 13, 13, 13, 14, 15, 15, 15, 15] 1047.6381611824036


In [16]:
t = (48, 7, 2**33-1)
start = time()
print(2*t[0], t[1]+1, hex(t[2]), test(*t, strong=False), time()-start)

96 8 0x1ffffffff [25, 24, 25, 24, 24, 24, 25, 25, 24, 25, 24, 24, 24, 24, 23, 22, 21, 20, 20, 20, 19, 18, 17, 17, 16, 17, 16, 15, 14, 14, 14, 14, 14, 13, 13, 13, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 17, 18, 18, 18, 18, 18, 18, 18, 19, 18, 18, 18, 18, 18, 18, 18, 17, 17, 16, 16, 16, 16, 15, 14, 13, 12, 12, 12, 11, 10, 9, 9, 8, 9, 8, 7, 7, 7, 8, 9, 10, 10, 11, 12, 13, 14, 15, 16] 4912.761736869812


In [17]:
t = (48, 20, 2**95-1)
start = time()
print(2*t[0], t[1]+1, hex(t[2]), test(*t, strong=False), time()-start)

96 21 0x7fffffffffffffffffffffff [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1] 129.90634417533875


In [18]:
t = (48, 21, 2**95-1)
start = time()
print(2*t[0], t[1]+1, hex(t[2]), test(*t, strong=False), time()-start)

96 22 0x7fffffffffffffffffffffff [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 56.9312903881073


## Simon-128

In [21]:
t = (64, 6, 2**17-1)
start = time()
print(2*t[0], t[1]+1, hex(t[2]), test(*t, strong=False), time()-start)

128 7 0x1ffff [16, 16, 16, 16, 16, 16, 16, 16, 16, 15, 14, 13, 12, 11, 10, 10, 9, 9, 8, 7, 6, 7, 8, 9, 10, 9, 10, 10, 11, 12, 13, 13, 13, 13, 13, 14, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 16, 16, 16, 16, 16, 16, 16, 15, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 9, 10, 9, 10, 10, 11, 12, 13, 13, 13, 13, 13, 14, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17] 4099.448934555054


In [22]:
t = (64, 7, 2**33-1)
start = time()
print(2*t[0], t[1]+1, hex(t[2]), test(*t, strong=False), time()-start)

128 8 0x1ffffffff [31, 30, 31, 31, 31, 31, 31, 31, 30, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 18, 17, 17, 16, 15, 14, 14, 14, 14, 14, 13, 13, 13, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 25, 26, 26, 27, 28, 29, 29, 29, 29, 29, 30, 31, 31, 31, 31, 29, 29, 28, 28, 29, 29, 29, 29, 29, 28, 28, 28, 27, 26, 25, 24, 23, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 10, 9, 9, 8, 7, 7, 7, 8, 9, 10, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 25, 26, 26, 27, 28, 29, 29] 32500.25850367546


In [23]:
t = (64, 24, 2**127-1)
start = time()
print(2*t[0], t[1]+1, hex(t[2]), test(*t, strong=False), time()-start)

128 25 0x7fffffffffffffffffffffffffffffff [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1] 239.26971769332886


In [24]:
t = (64, 25, 2**127-1)
start = time()
print(2*t[0], t[1]+1, hex(t[2]), test(*t, strong=False), time()-start)

128 26 0x7fffffffffffffffffffffffffffffff [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 189.58946323394775
